## Анастасия Куканова, A3200
### Задание 1. Градиентный спуск

#### Исходный код

In [1]:
from functools import reduce
from random import random
import math
import numpy as np


def generate_point(lower, upper):
    return np.array(tuple((l + random() * (u - l)) for l, u in zip(lower, upper)))


def in_range(point, lower, upper):
    return all(tuple(l <= comp <= u for comp, l, u in zip(point, lower, upper)))


def constant_step(step):
    while True:
        yield step


def decreasing_step(initial, coefficient):
    step = initial
    while True:
        yield step
        step *= coefficient


def descent(lower, upper, func, grad, step_gen, *step_args, normalize_grad=False, epsilon=0.01,
            print_result=False, return_step_count=False):
    step = step_gen(*step_args)
    step_count = 1
    current_point = generate_point(lower, upper)
    if print_result:
        print("Start: (", reduce(lambda a, b: "{}, {}".format(a, b), current_point), ")", sep="")
    while True:
        lam = next(step)
        g = grad(current_point)
        if normalize_grad:
            g /= np.linalg.norm(g)
        step_count += 1
        while True:
            next_point = current_point - lam * g
            if in_range(next_point, lower, upper):
                break
            else:
                lam /= 2.0
        if step_count > 100000 or np.linalg.norm(current_point - next_point) < epsilon:
            if print_result:
                print("Steps done:", step_count)
                print("Result: f(", reduce(lambda a, b: "{}, {}".format(a, b), next_point), ") =  ",
                      func(next_point)[0], sep="")
            if return_step_count:
                return next_point, func(next_point), step_count
            else:
                return next_point, func(next_point)
        else:
            current_point = next_point


def descent_optimal_step(lower, upper, func, grad, normalize_grad=False, epsilon=0.01,
                         print_result=False, return_step_count=False):
    step_count = 1
    dich_step_count = 0
    current_point = generate_point(lower, upper)
    if print_result:
        print("Start: (", reduce(lambda a, b: "{}, {}".format(a, b), current_point), ")", sep="")
    lam_max = np.linalg.norm(upper - lower)
    while True:
        g = grad(current_point)
        if normalize_grad:
            g /= np.linalg.norm(g)
        step_count += 1
        while True:
            lam, dsc = dichotomy(np.array([0]), np.array([lam_max]), lambda lam_: func(current_point - lam_ * g),
                                 epsilon=epsilon * 0.1)
            next_point = current_point - lam * g
            if in_range(next_point, lower, upper):
                dich_step_count += dsc
                break
            else:
                lam_max /= 2.0
        if np.linalg.norm(current_point - next_point) < epsilon:
            if print_result:
                print("Steps done:", step_count)
                print("Result: f(", reduce(lambda a, b: "{}, {}".format(a, b), next_point), ") =  ",
                      func(next_point)[0], sep="")
            if return_step_count:
                return next_point, func(next_point), step_count, dich_step_count
            else:
                return next_point, func(next_point)
        else:
            current_point = next_point


def dichotomy(left, right, func, epsilon=0.001):
    step_count = 0
    while right - left >= 2 * epsilon:
        step_count += 1
        mid = (left + right) / 2.0
        delta = random() * (right - left) / 2.0
        new_left = mid - delta
        new_right = mid + delta
        if func(new_left) > func(new_right):
            left = new_left
        else:
            right = new_right
    return (right + left) / 2, step_count


def monte_carlo_hybrid(opt_method, *args, tries=100, **kwargs):
    arg_min, _min = opt_method(*args, **kwargs)
    for _ in range(tries - 1):
        point, value = opt_method(*args, **kwargs)
        if value < _min:
            arg_min, _min = point, value
    return arg_min, _min


#### Вспомогательные функции для анализа экспериментов

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt

%matplotlib inline


def stats(results, true_min=0, mode=""):
    print("Average distance to the minimum: ", end="")
    print(sum([np.linalg.norm(res[0] - true_min) for res in results]) / len(results))
    if mode != "monte":
        print("Average steps done: ", end="")
        print(sum([res[2] for res in results]) / len(results))
        if mode == "opt":
            print("Average dichotomy steps done: ", end="")
            print(sum([res[3] for res in results]) / len(results))


def global_vs_local(results, true_min=0):
    global_count = 0
    local_count = 0
    for res in results:
        if np.linalg.norm(res[0] - true_min) < 0.01:
            global_count += 1
        else:
            local_count += 1
    print("Stopped at local minimum: ", local_count / len(results) * 100, "% of times", sep="")
    print("Stopped at global minimum: ", global_count / len(results) * 100, "% of times", sep="")


#### Исследуемые функции
Функции Розенброка:

$ f(x_1, x_2, ..., x_n) = \sum\limits_{i=1}^{n-1} [(1-x_i)^2 + 100(x_{i+1} - x_i^2)^2] $

$ \nabla f(x_1, x_2, ..., x_n) = \begin{pmatrix}
-2(1-x_1) - 400x_1(x_2 - x_1^2) \\
200(x_2 - x_1^2) - 2(1-x_2) -400x_2(x_3 - x_2^2) \\
... \\
200(x_k - x_{k-1}^2) - 2(1-x_k) -400x_k(x_{k+1} - x_k^2) \\
... \\
200(x_{n-1} - x_{n-2}^2) - 2(1-x_{n-1}) -400x_{n-1}(x_{n} - x_{n-1}^2) \\
200(x_n - x_{n-1}^2)
\end{pmatrix} $

![rosenbrock](https://psv4.vk.me/c414626/u182545048/docs/2be9a0c9e5e5/figure_3.png?extra=6X5Gpv6qtohbbmmCw30Ge9glfCkg_lqljHvKFkNbDLa8CiSr3ogTIVfKVjy12oIzaEKIwCx1QVT_Ac4vcQcFvd5M97vbCmxErAJd3AXLy3gj_TnvyBtRvYsEG5zWP_Kz6kdFrd-iUIUY)

In [3]:
def rosenbrock(vec):
    return np.array([sum(map(lambda a: (1.0 - a[0]) ** 2 + 100.0 * (a[1] - a[0] ** 2) ** 2, zip(vec, vec[1:])))])


def rosenbrock_grad(vec):
    n = len(vec)
    grad = [-2 * (1 - vec[0]) - 400 * vec[0] * (vec[1] - vec[0] ** 2)]
    for k in range(1, n - 1):
        grad.append(200 * (vec[k] - vec[k - 1] ** 2) - 2 * (1 - vec[k] - 400 * vec[k] * (vec[k + 1] - vec[k] ** 2)))
    grad.append(200 * (vec[n - 1] - vec[n - 2] ** 2))
    return np.array(grad)


Параболоид. Используется как легкая для нахождения минимума функция в целях проверки работоспособности алгоритмов, а также для демонстрации проблемы градиентного спуска в случае отсутствия в исследуемом множестве точек локального минимума:

$ f(x,y) = x^2 + y^2 $

$ \nabla f(x,y) = \begin{pmatrix} 2x \\ 2y \end{pmatrix} $

![paraboloid](https://psv4.vk.me/c404626/u182545048/docs/6cb1453c8487/figure_5.png?extra=jYsKGnAtgW24NkSlAC0pAfHSeNuR_qPMZ6Jo5BttdN8EOS5-pOH2O9q7ofiFeeAgAZJp4ry3igdF1yo34a0LtKhyHjuhFqlRZYrK9iDjIotHM9NLfoRzp_VbvJu2SBKSqna1KzewOahk)

In [4]:
def paraboloid(vec):
    (x, y) = vec
    return np.array([x ** 2 + y ** 2])


def paraboloid_grad(vec):
    (x, y) = vec
    return np.array([2 * x, 2 * y])


Данная функция используется для демонстрации проблемы достижения локального минимума на пути к глобальному, так как она имеет континуум точек локального минимума, окружающих точку глобального минимума:

$ f(x,y) = 15e^{-(x^2+y^2)}(x^2+y^2)+0.4^6(x^2+y^2)^3 $

$ \nabla f(x,y) = \begin{pmatrix}
2x(15e^{-(x^2+y2)}(1-(x^2+y^2)) + 0.4^6 \cdot 3(x^2+y^2)^2) \\
2y(15e^{-(x^2+y2)}(1-(x^2+y^2)) + 0.4^6 \cdot 3(x^2+y^2)^2)
\end{pmatrix} $

![tricky](https://psv4.vk.me/c414626/u182545048/docs/ce23cf0c424c/figure_1.png?extra=3QspGl2V7EXZA-zlDq7QqEQdwWeONXTb3XiRrrxI5-rPDPJXrBZ4wXCwxg3IL2Reh6R4uMRRR4GldNw3T6fEYcr2IIlngdcwHJ1fLmjLAFdrLBFESGlYKLATL7j06DwLuKDhnGvz0KEY)

In [5]:
def tricky(vec):
    (x, y) = vec
    return np.array(15.0 * math.exp(-(x ** 2 + y ** 2)) * (x ** 2 + y ** 2 - 0.4) + (0.4 ** 6) * (x ** 2 + y ** 2) ** 3)


def tricky_grad(vec):
    (x, y) = vec
    return np.array([
        (2 * x) * (15.0 * math.exp(-(x ** 2 + y ** 2)) * (1.0 - (x ** 2 + y ** 2 - 0.4)) +
                   3 * (0.4 ** 6) * (x ** 2 + y ** 2)),
        (2 * y) * (15.0 * math.exp(-(x ** 2 + y ** 2)) * (1.0 - (x ** 2 + y ** 2 - 0.4)) +
                   3 * (0.4 ** 6) * (x ** 2 + y ** 2)),
    ])


#### Градиентный спуск с постоянным шагом

Градиентный спуск с постоянным шагом 0.01 неплохо справляется с задачей на параболоиде. Уменьшение величины $ \varepsilon $
на порядок приводит к уменьшению среднего расстояния между результатом и истинным минимумом также на порядок:

In [6]:
__lower__ = np.array([-5, -5])
__upper__ = np.array([5, 5])
__eps__ = 0.001

for _ in range(3):
    print("epsilon =", __eps__)
    stats([descent(__lower__, __upper__, paraboloid, paraboloid_grad, constant_step, *[0.01], epsilon=__eps__,
                   return_step_count=True) for _ in range(50)])
    print()
    __eps__ *= 0.1


epsilon = 0.001
Average distance to the minimum: 0.048495554615
Average steps done: 210.94

epsilon = 0.0001
Average distance to the minimum: 0.00485881037756
Average steps done: 320.68

epsilon = 1e-05
Average distance to the minimum: 0.000485736945462
Average steps done: 444.52



Однако, на функции Розенброка градиентный спуск с шагом 0.01 не сойдется:

In [7]:
__lower__ = np.array([0, 0])
__upper__ = np.array([2, 2])

stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, constant_step, *[0.01], epsilon=0.001,
               return_step_count=True) for _ in range(3)], np.array([1, 1]))


Average distance to the minimum: 1.12643589782
Average steps done: 100001.0


При уменьшении шага до 0.001 метод сходится, но со значительной ошибкой. Уменьшение $ \varepsilon $ приводит к улучшению результата ценой значительного увеличения числа шагов:

In [8]:
__lower__ = np.array([0, 0])
__upper__ = np.array([2, 2])
__true_min__ = np.array([1, 1])
__eps__ = 0.0001

for _ in range(3):
    print("epsilon =", __eps__)
    stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, constant_step, *[0.001], epsilon=__eps__,
                   return_step_count=True) for _ in range(50)], __true_min__)
    print()
    __eps__ *= 0.1


epsilon = 0.0001
Average distance to the minimum: 0.217042248438
Average steps done: 1533.14

epsilon = 1e-05
Average distance to the minimum: 0.0249625216827
Average steps done: 6629.5

epsilon = 1.0000000000000002e-06
Average distance to the minimum: 0.00250155541479
Average steps done: 11665.4



На трехмерной функции Розенброка метод работает плохо, не приближаясь к точке минимума за десятки тысяч шагов:

In [9]:
__lower__ = np.array([0, 0, 0])
__upper__ = np.array([2, 2, 2])
__true_min__ = np.array([1, 1, 1])

stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, constant_step, *[0.01], epsilon=0.0001,
               return_step_count=True) for _ in range(50)], __true_min__)


Average distance to the minimum: 1.30916142764
Average steps done: 44004.98


#### Градиентный спуск с уменьшающимся шагом

На параболоиде отлично справляется с задачей при любых адекватных параметрах (не слишком маленький начальный шаг, не слишком быстрое уменьшение шага): за небольшое число шагов достигается высокая точность.

In [10]:
__lower__ = np.array([-5, -5])
__upper__ = np.array([5, 5])
__eps__ = 0.0001
stats([descent(__lower__, __upper__, paraboloid, paraboloid_grad, decreasing_step, *[2, 0.95], epsilon=__eps__,
               return_step_count=True) for _ in range(50)])
print()
stats([descent(__lower__, __upper__, paraboloid, paraboloid_grad, decreasing_step, *[1, 0.97], epsilon=__eps__,
               return_step_count=True) for _ in range(50)])
print()
stats([descent(__lower__, __upper__, paraboloid, paraboloid_grad, decreasing_step, *[1.5, 0.9], epsilon=__eps__,
               return_step_count=True) for _ in range(50)])


Average distance to the minimum: 2.42332172569e-06
Average steps done: 28.04

Average distance to the minimum: 6.32207992185e-06
Average steps done: 19.6

Average distance to the minimum: 6.68463005753e-06
Average steps done: 14.02


На сложных для оптимизации функциях, в частности, на функции Розенброка, метод становится крайне чувствителен к подбору параметров, и выдает крайне неточные результаты при очень небольших значениях $ \varepsilon $:

In [11]:
__lower__ = np.array([0, 0])
__upper__ = np.array([2, 2])
__true_min__ = np.array([1, 1])
__eps__ = 0.000001

stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[2.7, 0.995], epsilon=__eps__,
               return_step_count=True) for _ in range(50)], __true_min__)
print()
stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[2.7, 0.99], epsilon=__eps__,
               return_step_count=True) for _ in range(50)], __true_min__)
print()
stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[2.6, 0.98], epsilon=__eps__,
               return_step_count=True) for _ in range(50)], __true_min__)
print()
stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[2.3, 0.95], epsilon=__eps__,
               return_step_count=True) for _ in range(50)], __true_min__)


Average distance to the minimum: 0.513264293185
Average steps done: 2586.3

Average distance to the minimum: 0.689497062207
Average steps done: 1365.86

Average distance to the minimum: 0.876957449092
Average steps done: 669.98

Average distance to the minimum: 1.01492501447
Average steps done: 286.0


К значительному улучшению результатов использования данного метода при сохранении остальных параметров приводит нормализация вектора градиента. В случае с функцией Розенброка, градиент которой в окрестности точки $(1, 1)$ невелик по модулю, его нормализация гарантирует, что процесс не закончится слишком быстро. В целом, использование нормированного градиента может облегчить подбор параметров, в частности, для функций, быстро возрастающих при небольшом удалении от точки минимума.

In [12]:
__lower__ = np.array([0, 0])
__upper__ = np.array([2, 2])
__true_min__ = np.array([1, 1])
__eps__ = 0.000001

stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[2.7, 0.995], epsilon=0.000001,
               normalize_grad=True, return_step_count=True) for _ in range(50)], __true_min__)
print()
stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[2.7, 0.99], epsilon=0.000001,
               normalize_grad=True, return_step_count=True) for _ in range(50)], __true_min__)
print()
stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[2.6, 0.98], epsilon=0.000001,
               normalize_grad=True, return_step_count=True) for _ in range(50)], __true_min__)
print()
stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[2.3, 0.95], epsilon=0.000001,
               normalize_grad=True, return_step_count=True) for _ in range(50)], __true_min__)


Average distance to the minimum: 0.0466913077546
Average steps done: 2839.44

Average distance to the minimum: 0.0457466897647
Average steps done: 1417.56

Average distance to the minimum: 0.144399775371
Average steps done: 705.12

Average distance to the minimum: 0.530960538034
Average steps done: 282.4


На трехмерной функции Розеброка метод проявляет себя плохо (хотя нельзя исключать, что я подобрала слишком плохие параметры):

In [13]:
__lower__ = np.array([0, 0, 0])
__upper__ = np.array([2, 2, 2])
__true_min__ = np.array([1, 1, 1])
__eps__ = 0.000001

stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[5, 0.99], epsilon=__eps__,
               normalize_grad=True, return_step_count=True) for _ in range(50)], __true_min__)
print()
stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[4, 0.99], epsilon=__eps__,
               normalize_grad=True, return_step_count=True) for _ in range(50)], __true_min__)
print()
stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[3.5, 0.98], epsilon=__eps__,
               normalize_grad=True, return_step_count=True) for _ in range(50)], __true_min__)
print()
stats([descent(__lower__, __upper__, rosenbrock, rosenbrock_grad, decreasing_step, *[3, 0.95], epsilon=__eps__,
               normalize_grad=True, return_step_count=True) for _ in range(50)], __true_min__)


Average distance to the minimum: 1.23453611505
Average steps done: 25.52

Average distance to the minimum: 1.25295156815
Average steps done: 22.76

Average distance to the minimum: 1.2063140146
Average steps done: 18.26

Average distance to the minimum: 1.20175153607
Average steps done: 24.56


#### Градиентный спуск с оптимальным шагом

На параболоиде справляется с задачей отлично.

In [14]:
__lower__ = np.array([-5, -5])
__upper__ = np.array([5, 5])

stats([descent_optimal_step(__lower__, __upper__, paraboloid, paraboloid_grad, epsilon=0.0001,
                            return_step_count=True) for _ in range(50)], mode="opt")


Average distance to the minimum: 1.34188025395e-10
Average steps done: 3.0
Average dichotomy steps done: 87.66


На функции Розенброка возникают проблемы как с точностью, так и довольно большим количеством шагов. Нормализация вектора улучшает ситуацию довольно значительно, но недостаточно. Однако, следует учитывать, что результаты с точностью того же порядка получались градиентным спуском с уменьшающимся шагом с параметрами, совсем немного отличающихся от тех, что давали лучший результат, при значении $ \varepsilon $ на два порядка меньше, чем в следующем эксперименте. Таким образом, данный метод справляется с задачей хорошо, и градиентный спуск с уменьшающимся шагом делает это лучше только при очень тщательном подборе параметров, которого не требуется для данного метода:

In [15]:
__lower__ = np.array([0, 0])
__upper__ = np.array([2, 2])
__true_min__ = np.array([1, 1])
__eps__ = 0.0001

stats([descent_optimal_step(__lower__, __upper__, rosenbrock, rosenbrock_grad, epsilon=__eps__,
                            return_step_count=True) for _ in range(50)], __true_min__, mode="opt")
print()
stats([descent_optimal_step(__lower__, __upper__, rosenbrock, rosenbrock_grad, epsilon=__eps__,
                            normalize_grad=True, return_step_count=True) for _ in range(50)], __true_min__, mode="opt")


Average distance to the minimum: 0.0715177648239
Average steps done: 1004.34
Average dichotomy steps done: 35069.32

Average distance to the minimum: 0.0453260702239
Average steps done: 294.8
Average dichotomy steps done: 10371.76


На трехмерной функции Розенброка метод дает низкую степень точности, которая не повышается уменьшением величины $ \varepsilon $:

In [16]:
__lower__ = np.array([0, 0, 0])
__upper__ = np.array([2, 2, 2])
__true_min__ = np.array([1, 1, 1])
__eps__ = 0.0001

for _ in range(3):
    print("epsilon =", __eps__)
    stats([descent_optimal_step(__lower__, __upper__, rosenbrock, rosenbrock_grad, epsilon=__eps__,
                                normalize_grad=True, return_step_count=True) for _ in range(50)], __true_min__, mode="opt")
    print()
    __eps__ *= 0.1


epsilon = 0.0001
Average distance to the minimum: 0.951456578235
Average steps done: 8.24
Average dichotomy steps done: 216.02

epsilon = 1e-05
Average distance to the minimum: 0.857404962326
Average steps done: 6.14
Average dichotomy steps done: 199.32

epsilon = 1.0000000000000002e-06
Average distance to the minimum: 0.811984219188
Average steps done: 12.34
Average dichotomy steps done: 474.9



Общей проблемой для всех модификаций градиентного спуска является то, что если процесс на пути к точке глобального минимума попадет в локальный, то он не выходит из нее. Это хорошо видно на последней из выбранных мной для рассмотрения функций, имеющей континуум локальных минимумов, окружающих глобальный:

In [17]:
__lower__ = np.array([-3, -3])
__upper__ = np.array([3, 3])
__eps__ = 0.0001

print("Constant step")
global_vs_local([descent(__lower__, __upper__, tricky, tricky_grad, constant_step, *[0.01], epsilon=__eps__,
                         return_step_count=True) for _ in range(100)])
print("\nDecreasing step")
global_vs_local([descent(__lower__, __upper__, tricky, tricky_grad, decreasing_step, *[9, 0.99], epsilon=__eps__,
                         normalize_grad=True, return_step_count=True) for _ in range(10)])
print("\nOptimal step")
global_vs_local([descent_optimal_step(__lower__, __upper__, tricky, tricky_grad, epsilon=__eps__,
                                      return_step_count=True) for _ in range(100)])


Constant step
Stopped at local minimum: 90.0% of times
Stopped at global minimum: 10.0% of times

Decreasing step
Stopped at local minimum: 10.0% of times
Stopped at global minimum: 90.0% of times

Optimal step
Stopped at local minimum: 72.0% of times
Stopped at global minimum: 28.000000000000004% of times


Гибридный подход, совмещающий градиентный спуск с методом Монте-Карло, решает эту проблему:

In [18]:
__lower__ = np.array([-3, -3])
__upper__ = np.array([3, 3])
__eps__ = 0.0001

print("Constant step")
global_vs_local([monte_carlo_hybrid(descent, __lower__, __upper__, tricky, tricky_grad, constant_step, *[0.01],
                                    tries=100, epsilon=__eps__) for _ in range(10)])
print("\nDecreasing step")
global_vs_local([monte_carlo_hybrid(descent, __lower__, __upper__, tricky, tricky_grad, decreasing_step, *[9, 0.99],
                                    tries=100, normalize_grad=True, epsilon=__eps__) for _ in range(10)])
print("\nOptimal step")
global_vs_local([monte_carlo_hybrid(descent_optimal_step, __lower__, __upper__, tricky, tricky_grad, tries=100,
                                    epsilon=__eps__) for _ in range(10)])


Constant step
Stopped at local minimum: 0.0% of times
Stopped at global minimum: 100.0% of times

Decreasing step
Stopped at local minimum: 0.0% of times
Stopped at global minimum: 100.0% of times

Optimal step
Stopped at local minimum: 0.0% of times
Stopped at global minimum: 100.0% of times


В целом, использование гибридного алгоритма значительно улучшает результаты:

In [20]:
__lower__ = np.array([0, 0])
__upper__ = np.array([2, 2])
__true_min__ = np.array([1, 1])

stats([monte_carlo_hybrid(descent_optimal_step, __lower__, __upper__, rosenbrock, rosenbrock_grad, tries=100,
                                epsilon=0.0001, normalize_grad=True) for _ in range(10)], __true_min__, mode="monte")


Average distance to the minimum: 0.000224994110241


In [21]:
__lower__ = np.array([0, 0, 0])
__upper__ = np.array([2, 2, 2])
__true_min__ = np.array([1, 1, 1])

stats([monte_carlo_hybrid(descent_optimal_step, __lower__, __upper__, rosenbrock, rosenbrock_grad, tries=100,
                                epsilon=0.0001, normalize_grad=True) for _ in range(10)], __true_min__, mode="monte")


Average distance to the minimum: 0.712892252433


Однако, это происходит ценой значительного увеличения числа операций.

Другая проблема градиентного спуска проявляет себя в случае, если на исследуемом множестве функция не имеет точек локального минимума. Двигаясь в направлении ближайшей такой точки, алгоритм "упирается" в границу и продолжает попытки совершать небольшие шаги в том направлении, пока не выполнится условие завершения работы. Это легко продемонстировать на следующем примере:

In [22]:
__lower__ = np.array([1, 2])
__upper__ = np.array([3, 4])
__true_min__ = __lower__

stats([descent_optimal_step(__lower__, __upper__, paraboloid, paraboloid_grad, epsilon=0.0001, normalize_grad=True,
                                return_step_count=True) for _ in range(50)], __true_min__, mode="opt")


Average distance to the minimum: 0.612220226098
Average steps done: 9.22
Average dichotomy steps done: 154.86


#### Выводы
1. Градиентный спуск с уменьшающимся шагом и градиентный спуск с оптимальным шагом проявляют себя лучше, если нормализовать вектор градиента.
2. Метод с уменьшающимся шагом может проявлять себя лучше метода с оптимальным шагом при хорошем подборе параметров. Однако, метод с оптимальным шагом не требует подбирать параметры вручную, что, безусловно, является его преимуществом.
3. Проблема достижения локального минимума на пути к глобальному решается совместным использованием градиентного спуска и некоторого стохастического метода, например, метода Монте-Карло. Использование гибридного алгоритма в целом приводит к повышению точности результата, в том числе на сложных для нахождения минимума функциях.
4. Существует проблема с нахождением минимума на множестве, где нет точек локального минимума. По всей видимости, для ее разрешения необходимо проверять граничные точки.
5. Градиентный спуск проявляет себя хуже с увеличением размерности пространства.